In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms, utils, models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from torch.autograd import Variable
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import torch
import numpy
import re
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataloader import default_collate
from transformers import XLMRobertaTokenizer, XLMRobertaXLModel
from transformers import XLNetTokenizer, XLNetModel
import random
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision.ops.focal_loss as fl
import tensorflow_addons as tfa
import gc

In [2]:
TRAIN1_DIR = '/kaggle/input/foody-data-after-pre/pre_train.csv'
TRAIN2_DIR = '/kaggle/input/foody-data-after-pre/data2_processed.csv'
IMAGE1_DIR = '/kaggle/input/foody-image/images/hihi{}_0__image.jpg'
IMAGE2_DIR = '/kaggle/input/image-crawl/image2/hihi{}_0__image.jpg'
IMAGE_TEST = '/kaggle/input/foody-image-test/image_for_test/hihi{}_0__image.jpg'
TEST_DIR = '/kaggle/input/foody-data-after-pre/pre_test.csv'
VAL_DIR = '/kaggle/input/foody-data-after-pre/val_labelled.csv'

In [3]:
torch.manual_seed(25)
np.random.seed(25)
random.seed(25)

cudnn.benchmark = True
cudnn.deterministic = True

In [4]:
df = pd.read_csv(TRAIN1_DIR)

In [5]:
df = df.drop("Unnamed: 0", axis = 1)

In [6]:
df['Rating'] = df['Rating'].astype(int)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
list_image_paths = []

for i in range(df.shape[0]):
    list_image_paths.append(IMAGE1_DIR.format(i))

In [9]:
df['image_paths'] = list_image_paths

In [10]:
df

,len,Comment,Rating,image_paths
0,13,xôi dẻo đồ_ăn đậm vị hộp xôi được lót lá trông...,1,/kaggle/input/foody-image/images/hihi0_0__imag...
1,53,gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,0,/kaggle/input/foody-image/images/hihi1_0__imag...
2,48,thời_tiết lạnh như này cả nhà rủ nhau đến lega...,1,/kaggle/input/foody-image/images/hihi2_0__imag...
3,57,em có đọc review thấy mng bảo trà sữa nướng đề...,0,/kaggle/input/foody-image/images/hihi3_0__imag...
4,92,đồ_ăn rất ngon nhà_hàng cũng rất đẹp tất_cả đề...,1,/kaggle/input/foody-image/images/hihi4_0__imag...
...,...,...,...,...
9066,30,thực_sự mà nói thấy mọi người đánh_giá nhiều q...,1,/kaggle/input/foody-image/images/hihi9066_0__i...
9067,134,lẩu thái tômyum đc 19 lò_đúc hbt hn theo đánh_...,1,/kaggle/input/foody-image/images/hihi9067_0__i...
9068,45,ngay từ lúc đầu_tiên bước vào nhà_hàng đã được...,1,/kaggle/input/foody-image/images/hihi9068_0__i...
9069,17,đặt ăn thử mà thấy ngón cá emo rất mê sẽ còn ủ...,1,/kaggle/input/foody-image/images/hihi9069_0__i...


In [11]:
df['Comment'] = df['Comment'].str.strip()

In [12]:
df = df[df['Comment']  != 'None']

In [13]:
# from sklearn.model_selection import train_test_split
# df_train, df_test = train_test_split(df, test_size=0.15, random_state=42)

df_train = df
df_val = pd.read_csv(VAL_DIR)

In [14]:
df_val['Rating'] = df_val['Rating'].astype(int)

In [15]:
list_image_paths = []

for i in range(df_val.shape[0]):
    list_image_paths.append(IMAGE_TEST.format(i))

In [16]:
df_val['image_paths'] = list_image_paths

In [17]:
df_2 = pd.read_csv(TRAIN2_DIR)
df_2 = df_2.drop("Unnamed: 0", axis = 1)

In [18]:
df_2['Rating'] = df_2['Rating'].astype(int)

In [19]:
list_image_paths = []

for i in range(df_2.shape[0]):
    list_image_paths.append(IMAGE2_DIR.format(i))

In [20]:
df_2['image_paths'] = list_image_paths

In [21]:
df_0_2 = df_2[df_2['Rating'] == 0].sample(1450)

In [22]:
df_0_2['Rating'].value_counts()

0    1450
Name: Rating, dtype: int64

In [23]:
df_1_2 = df_2[df_2['Rating'] == 1].sample(1000)

In [24]:
df_train = pd.concat([df_train[['Comment','image_paths','Rating']],
                df_0_2[['Comment','image_paths','Rating']],
                df_1_2[['Comment','image_paths','Rating']]
               ])

In [25]:
df_train.shape

(11519, 3)

In [26]:
df_val['Rating'].value_counts()

1    4003
0    1100
Name: Rating, dtype: int64

In [27]:
df_val['Rating'] = df_val['Rating'].astype(int)

In [28]:
from sklearn.utils import shuffle

In [29]:
df_train = df_train.drop_duplicates(['Comment'])

In [30]:
df_train['Rating'].value_counts()

1    8122
0    3348
Name: Rating, dtype: int64

### model

In [31]:
data_transform_train = transforms.Compose([
        transforms.Resize((64,64)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

data_transform_test = transforms.Compose([
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])


In [32]:
class MultimodalDataset(Dataset):

    def __init__(self, img_dir, labels, text, transform=None):
        self.labels= labels
        self.img_dir = img_dir
        self.transform = transform
        self.text = text

    def __len__(self):
        return len(self.img_dir)

    def __getitem__(self, idx):
        image = Image.open(self.img_dir[idx]).convert('RGB')
        label = self.labels[idx]
        text = self.text[idx]

        if self.transform:
            image = self.transform(image)
        

        return [text, image, label]

In [33]:
from tqdm import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

class Multimodal_Bert_Vgg(nn.Module):
    def __init__(self):
        super(Multimodal_Bert_Vgg, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
        self.vgg = models.vgg11(pretrained=True).to(device)
        self.vgg.classifier[6] = nn.Linear(4096, 256).to(device)

        self.linear2 = nn.Linear(768,256).to(device)
        
        self.classifier = nn.Sequential(
          nn.Dropout(0.2),
          nn.Linear(in_features = 512, out_features = 256, bias=True).to(device),
          nn.Dropout(0.1),
          nn.Linear(in_features = 256, out_features = 2, bias=True).to(device)  
        )
        
        self.criterior = nn.CrossEntropyLoss().to(device)
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        param_optimizer = list(self.bert.named_parameters())
        self.optimizer =optim.AdamW(
            [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            {'params': self.classifier.parameters(), 'lr': 8e-5},
            {'params': self.vgg.parameters(), 'lr': 2e-5},
            {'params': self.linear2.parameters(), 'lr': 8e-5},
            ],
            lr = 2e-5
        )

    def tokenize(self,v_text, images, label, phase = 'train'):
        v_tokenized = []
        max_len = 256
        for i_text in v_text:
            i_text = standardize_data(i_text)
            i_text = self.choose_sub_text(i_text)
            line = self.tokenizer.encode(i_text, max_length = max_len)
            v_tokenized.append(line)

        padded = numpy.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
        attention_mask = numpy.where(padded == 1, 0, 1)
        padded = torch.tensor(padded).to(torch.long)
        attention_mask = torch.tensor(attention_mask)
        
        target = torch.tensor(label) 
        print(target)
        if phase == 'train':
            data_tensor = MultimodalDataset(img_dir = images, 
                        labels = label, 
                        text = padded, 
                        transform = data_transform_train)
        else:
            data_tensor = MultimodalDataset(img_dir = images, 
                        labels = label, 
                        text = padded, 
                        transform = data_transform_test)
            
        train_loader = DataLoader(dataset = data_tensor, batch_size = 32)
        attention_mask = DataLoader(attention_mask, batch_size = 32)
        
        return train_loader, attention_mask
    
    def choose_sub_text(self, text):
        tmp = len(text.split())
        if tmp > 256:
            words = text.split()
            text = ' '.join(words[0:210]) + ' ' + ' '.join(words[-46:])
            
        return text
    
    def forward(self, input_ids, images, attention_mask):
        _, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
             return_dict=False
        )
        output = self.linear2(output)
        output2 = self.vgg(images)
        output = torch.cat((output, output2), 1)
        output = self.classifier(output)
        return output

In [34]:
def standardize_data(row):
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ") \
        .replace("  "," ").replace("  "," ") \
        .replace(')','').replace('(','') \
        .replace("—",' ').replace("\n",' '). replace("\t",' ') \
        .replace('   ', ' ').replace('  ', ' ')
            
    return row

In [35]:
def train_model(bert, save_path, df_train, df_test, num_epochs=1, type_model = 'bert', check_fold = 1):
        text_train = df_train['Comment'].to_list()
        image_paths_train = df_train['image_paths'].to_list()
        label_train = df_train['Rating'].to_list()
        text_test = df_test['Comment'].to_list()
        image_paths_test = df_test['image_paths'].to_list()
        label_test = df_test['Rating'].to_list()
        Data_train, attention_train = bert.tokenize(text_train, image_paths_train, label_train, 'train')
        Data_test, attention_test = bert.tokenize(text_test, image_paths_test, label_test, 'test')
        
        dataloader_dict = {
            'train' : Data_train,
            'test' : Data_test
        }
        
        check = 0
        
        
        for epoch in range(num_epochs + 1):
            if epoch == 0:
                print()
                continue
            print("Epoch {}/{} ".format(epoch, num_epochs))

            for phase in ['train', 'test']:
                if phase == 'train':
                    bert.train()
                    attention = attention_train
                else:
                    bert.eval()
                    attention = attention_test

                epoch_loss = 0.0
                epoch_corrects = 0

                for (inputs, images, labels), i_attention in tqdm(zip(dataloader_dict[phase], attention), position=0, leave=True):
                    inputs = inputs.to(device)
                    images = images.to(device)
                    labels = labels.to(device)
                    i_attention = i_attention.to(device)

                    bert.optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = bert(inputs,images, i_attention)

                        loss = bert.criterior(outputs, labels)

                        _, preds = torch.max(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            bert.optimizer.step()
                            
                            

                        epoch_loss += loss.item() * inputs.size(0)
                        epoch_corrects += torch.sum(preds == labels.data)
                        del outputs
                        torch.cuda.empty_cache()

                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
                
                print("{} Loss: {:.4f}  Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
                
                if check < epoch_acc and phase == 'test':
                    check = epoch_acc
                    torch.save(bert.state_dict(), "./full_{}_model_fold{}.pth".format(type_model, check_fold))

### train

In [36]:
df_train = shuffle(df_train)

In [37]:
mb = Multimodal_Bert_Vgg()
train_model(mb, './', df_train, df_val, num_epochs = 4, type_model = 'multimodal')
# train_model(model,"./", df_train, df_test, num_epochs=epoch,type_model =type_model,check_fold = check_fold)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

...

test Loss: 0.2146  Acc: 0.9342
Epoch 4/4 


359it [10:10,  1.70s/it]


train Loss: 0.1586  Acc: 0.9478


160it [02:27,  1.09it/s]

test Loss: 0.2349  Acc: 0.9285


In [38]:
def predict_multi(model, df_test):
    image_test = df_test['image_paths'].to_list()
    text_test = df_test['Comment'].to_list()

    tmp = np.zeros(len(text_test))
    net = model
    net.eval()
    Data_test, attention_test = net.tokenize(text_test, image_test, tmp, 'test')

    attention = attention_test

    list_tmp = []
    list_truth = np.array([])

    for (inputs, images, labels), i_attention in tqdm(zip(Data_test, attention), position=0, leave=True):
        inputs = inputs.to(device)
        images = images.to(device)
        labels = labels.to(device)
        i_attention = i_attention.to(device)
        
        with torch.no_grad():

            outputs = net(inputs,images, i_attention)

            list_tmp.append(outputs)

            _, preds = torch.max(outputs, 1)

            del outputs
            torch.cuda.empty_cache()
            gc.collect()

            list_truth = np.append(list_truth,preds.cpu().numpy())
    
    list_prob = []
    for i in range(len(list_tmp)):
        probs = F.softmax(list_tmp[i], dim=1) # assuming logits has the shape [batch_size, nb_classes]
        list_prob.append(probs)
    
    list_ans = []
    for i in list_prob:
        list_ans = np.append(list_ans,i.cpu().numpy()[:,1])
        
    return list_truth, list_ans

In [39]:
mb_new = Multimodal_Bert_Vgg()
w = torch.load('/kaggle/working/full_multimodal_model_fold1.pth')
mb_new.load_state_dict(w)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

In [40]:
import torch.nn.functional as F

list_predict,list_ans = predict_multi(mb_new, df_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)


160it [03:05,  1.16s/it]


### submit

In [41]:
df_sub = pd.read_csv(TEST_DIR)

In [42]:
list_image_paths = []

for i in range(df_sub.shape[0]):
    list_image_paths.append(IMAGE_TEST.format(i))

In [43]:
df_sub['image_paths'] = list_image_paths

In [44]:
import torch.nn.functional as F

b = Multimodal_Bert_Vgg()
w = torch.load('/kaggle/working/full_multimodal_model_fold1.pth')
b.load_state_dict(w)
list_predict, list_ans1 = predict_multi(b, df_sub)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples t

tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)


160it [03:06,  1.16s/it]


In [45]:
list_ans1

array([0.34242716, 0.36027771, 0.98473722, ..., 0.00424484, 0.94609958,
       0.97676325])

In [48]:
df_sub['Rating'] = list_ans1

In [49]:
df_sub[['RevId','Rating']].to_csv('submission_bert_vgg.csv',index = False)